In [1]:
import json
from src.resources.integration.fparser_python import *
from src.resources.integration.fparser import *
from src.flasking.sending_windows import *

In [2]:
from pathlib import Path
with open("./feroz_context.txt", 'r') as f:
    context = f.read()

In [3]:
summary = parse_summary_data(context)

In [4]:
segments = parse_segment_data(context)

In [5]:
# segments

In [6]:
filtered_segments = filter_redundant_actions(segments)

In [7]:
final_unique_segments = filter_redundant_executed_actions(filtered_segments)

In [8]:
chain = decomposer_prompt | ollama_llm.with_structured_output(AtomicsModel, method="json_schema")

In [9]:
final_parse = decompose_segments_with_atomic_actions(chain, final_unique_segments)

In [10]:
from src.langchain.agents.enhanced_ad_agent import *

Trying OLLAMA model 8b
OLLAMA model found


In [11]:
instruction = final_parse[0]['instruction']
ad = final_parse[0]['action_designator']

In [12]:
ad

{'component_id': 'onion_01',
 'component_information': {'name': 'onion',
  'id_number': '01',
  'component_type': 'food_item',
  'shape': 'spherical',
  'size': 'medium',
  'handle': 'no',
  'orientation': 'upright',
  'weight': 1},
 'executed_action': {'primary_action': {'action_name': 'picking_up',
   'hand': 'right_hand'},
  'secondary_action': {'action_name': 'placing', 'hand': 'right_hand'}},
 'grasp_descriptor': {'grasp_type': 'spherical',
  'contact_points': 'three_fingers',
  'holding_type': 'one_handed',
  'hand_orientation': 'top_to_bottom'},
 'environmental_factors': {'surface_conditions': 'flat_surfaces'}}

In [13]:
graph_output = []

In [14]:
final_parse_filtered = [final_parse[0]]

In [15]:
# final_parse[4]

In [16]:
final_parse_filtered

[{'instruction': 'Pick up a brown onion.',
  'action_designator': {'component_id': 'onion_01',
   'component_information': {'name': 'onion',
    'id_number': '01',
    'component_type': 'food_item',
    'shape': 'spherical',
    'size': 'medium',
    'handle': 'no',
    'orientation': 'upright',
    'weight': 1},
   'executed_action': {'primary_action': {'action_name': 'picking_up',
     'hand': 'right_hand'},
    'secondary_action': {'action_name': 'placing', 'hand': 'right_hand'}},
   'grasp_descriptor': {'grasp_type': 'spherical',
    'contact_points': 'three_fingers',
    'holding_type': 'one_handed',
    'hand_orientation': 'top_to_bottom'},
   'environmental_factors': {'surface_conditions': 'flat_surfaces'}}}]

In [17]:
for fp in final_parse_filtered:
    instruction = fp['instruction']
    ad = fp['action_designator']
    out = ad_graph.invoke({'instruction': instruction, 'context': ad})
    graph_output.append(out)

INSIDE ACTION NODE
type={'action': 'PickingUp'}
['PickingUp']
INSIDE UNIVERSAL NODE
cram_plan (perform (an action (type grab-object) (an object (type {obj_to_be_grabbed}){obj_to_be_grabbed_props}) (source (a location (on {location}){location_props}))))
----------
{
  "obj_to_be_grabbed": "a brown onion",
  "action_verb": "Pick up",
  "location": null
}
----------
fCONTEXT {'component_id': 'onion_01', 'component_information': {'name': 'onion', 'id_number': '01', 'component_type': 'food_item', 'shape': 'spherical', 'size': 'medium', 'handle': 'no', 'orientation': 'upright', 'weight': 1}, 'executed_action': {'primary_action': {'action_name': 'picking_up', 'hand': 'right_hand'}, 'secondary_action': {'action_name': 'placing', 'hand': 'right_hand'}}, 'grasp_descriptor': {'grasp_type': 'spherical', 'contact_points': 'three_fingers', 'holding_type': 'one_handed', 'hand_orientation': 'top_to_bottom'}, 'environmental_factors': {'surface_conditions': 'flat_surfaces'}}
{
  "action_verb": "Pick up"

In [18]:
for seg in segments:
    print("📡 Sending test data to 1st window...")
    send_data_to_window(1, seg)

📡 Sending test data to 1st window...
✅ Data sent to Window 1: {'status': 'success', 'window': 'window1'}
📡 Sending test data to 1st window...
✅ Data sent to Window 1: {'status': 'success', 'window': 'window1'}
📡 Sending test data to 1st window...
✅ Data sent to Window 1: {'status': 'success', 'window': 'window1'}
📡 Sending test data to 1st window...
✅ Data sent to Window 1: {'status': 'success', 'window': 'window1'}
📡 Sending test data to 1st window...
✅ Data sent to Window 1: {'status': 'success', 'window': 'window1'}
📡 Sending test data to 1st window...
✅ Data sent to Window 1: {'status': 'success', 'window': 'window1'}
📡 Sending test data to 1st window...
✅ Data sent to Window 1: {'status': 'success', 'window': 'window1'}
📡 Sending test data to 1st window...
✅ Data sent to Window 1: {'status': 'success', 'window': 'window1'}
📡 Sending test data to 1st window...
✅ Data sent to Window 1: {'status': 'success', 'window': 'window1'}
📡 Sending test data to 1st window...
✅ Data sent to Win

In [ ]:
for par in final_parse:
    print("📡 Sending test data to 2nd window...")
    send_data_to_window(2, par)

In [28]:
# graph_output

In [101]:
for out in graph_output:
    print("<UNK> Sending test data to 3rd window...")
    new_out = {}
    try:
        new_out['instruction'] = out['instruction']
        new_out['action_core'] = out['action_core']
        new_out['action_core_attributes'] = out['action_core_attributes']
        new_out['enriched_action_core_attributes'] = json.loads(out['enriched_action_core_attributes'])
        new_out['cram_plan'] = out['cram_plan_response']
        print("Parsed output normally")
    except:
        new_out['instruction'] = out['instruction']
        new_out['action_core'] = out['action_core']
        new_out['action_core_attributes'] = out['action_core_attributes']
        new_out['enriched_action_core_attributes'] = out['enriched_action_core_attributes']
        new_out['cram_plan'] = out['cram_plan_response']
        print("Parsed output with strings")

    send_data_to_window(3, new_out)

<UNK> Sending test data to 3rd window...
Parsed output normally
✅ Data sent to Window 3: {'status': 'success', 'window': 'window3'}


In [102]:
from src.langchain.models_graph import *
config = {"configurable" : {"thread_id" : 1}}

In [103]:
framenet_model = ""
flanagan = ""

In [104]:
len(graph_output)

1

In [105]:
for out in graph_output:
    print("<UNK> Sending test data to 4th window...")
    instruction = out['instruction']
    action_core = out['action_core']
    action_core_attributes = out['action_core_attributes']
    enriched_action_core_attributes = json.loads(out['enriched_action_core_attributes'])
    cram_plan = out['cram_plan_response']
    final_graph_state = model_graph.invoke({"instruction": instruction,
                                                 "action_core": action_core,
                                                 "enriched_action_core_attributes": enriched_action_core_attributes,
                                                 "cram_plan_response": cram_plan},config=config, stream_mode="updates")
    flanagan = model_graph.get_state(config).values["flanagan"]
    framenet_model = model_graph.get_state(config).values["framenet_model"]
    new_out = {}
    try:
        flanagan_json = json.loads(flanagan)
        framenet_model_json = json.loads(framenet_model)
        print("Parsed models output normally")
        new_out = {
            "framenet" : framenet_model_json,
            "flanagan" : flanagan_json
        }
    except:
        print("Parsed models output with strings")
        new_out = {
            "framenet" : framenet_model,
            "flanagan" : flanagan
        }

    send_data_to_window(4, new_out)

<UNK> Sending test data to 4th window...
INSIDE flanagan PREMOTION TOOL
INSIDE FRAMENET NODE
goal_definition=GoalDefinition(task='Peeling brown onion', semantic_annotation='TaskClass:Manipulation.Cutting', object=ObjectModel(id='brown_onion_01', type='PhysicalArtifact', properties=ObjectProperties(size=None, texture='smooth', material=None, fill_level=None, contents=None, hardness=None, friction_coefficient=None, elasticity=None, strain_limit=1000000000000000.0), expected_end_state=GoalState(conditions={'is_peeled': {'state': True}})), target=None, tool=ToolModel(id='knife_main', type='Tool', properties={'material': 'metal', 'edge': 'sharp', 'handle': 'single', 'position': 'on', 'surface': 'wooden', 'orientation': 'upright'})) predictive_model=PredictiveModel(expected_trajectory='StandardCuttingApproach', expected_force={'initial_N': 2.0, 'resistance_range_N': [1.5, 3.0]}, confidence_level=0.95, affordance_model={'tool_affords_cutting': True, 'object_affords_being_cut': True}) motion_p

In [106]:
print(flanagan)

{
  "task": "Peeling brown onion",
  "pre_motion_phase": {
    "goal_definition": {
      "task": "Peeling brown onion",
      "semantic_annotation": "TaskClass:Manipulation.Cutting",
      "object": {
        "id": "brown_onion_01",
        "type": "PhysicalArtifact",
        "properties": {
          "size": null,
          "texture": "smooth",
          "material": null,
          "fill_level": null,
          "contents": null,
          "hardness": null,
          "friction_coefficient": null,
          "elasticity": null,
          "strain_limit": 1000000000000000.0
        },
        "expected_end_state": {
          "conditions": {
            "is_peeled": {
              "state": true
            }
          }
        }
      },
      "target": null,
      "tool": {
        "id": "knife_main",
        "type": "Tool",
        "properties": {
          "material": "metal",
          "edge": "sharp",
          "handle": "single",
          "position": "on",
          "surface": "w

In [107]:
print(framenet_model)

{
  "framenet": "peeling",
  "frame": "Peeling",
  "lexical-unit": "peel.v",
  "core": {
    "agent": "robot",
    "theme_patient": "onion",
    "instrument": "knife",
    "source": "brown onion",
    "goal": "skin off the onion",
    "result": "onion without skin"
  },
  "peripheral": {
    "location": "wooden cutting board",
    "manner": "carefully",
    "direction": "downward",
    "time": "during preparation",
    "quantity": "one onion",
    "portion": "skin"
  }
}
